In [1]:
import torch
import os
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import cv2
import torchvision
from torchvision import transforms
import albumentations
import albumentations.pytorch
from torch.utils.tensorboard import SummaryWriter
import datetime

In [2]:
writer = SummaryWriter('runs/')

%load_ext tensorboard

2021-10-18 07:31:14.174949: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-18 07:31:14.174978: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
torch.manual_seed(111)
device = torch.device("cuda")
torch.cuda.manual_seed_all(111)

In [4]:
# Dataset
class Dataset(Dataset):
    def __init__(self, transform = None):
        super().__init__()
        
        self.img_dir = "/home/temp_1/kangsanha/malocclusion/data"
        self.filenames = os.listdir(self.img_dir)

        self.transform = transform

    def __getitem__(self, index):
        label = MakeLabel(self.filenames[index])
        img_path = os.path.join(self.img_dir, self.filenames[index])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGR -> RGB
        # albumentations 라이브러리를 사용할 것인데, PIL을 쓰면 변환이 안된다하여 cv2로 변환해준다.
        # img = np.asarray_chkfinite(Image.open(img_path))
        
        if img.dtype == np.uint8:
            img = img / 255.0
        
        #img = ToTensor(img)
        #print(type(img))
        #print(np.shape(img))
        #img = img.transpose((2,0,1)).astype(np.float32)
        if self.transform:
            aug = self.transform(image=img)
            img = aug['image']
        #img = img.values()
        
        #  print(np.shape(img)) # 검사코드
        # input = {'data' : img, 'label' : label}
        
        return img, label
        

    def __len__(self):
        
        return len(self.filenames)

    
def MakeLabel(filenames):
    input_img = filenames
    if "O" in input_img:
        label = 1#label = F.one_hot(torch.tensor(0), 2)
    else : label = 0#label = F.one_hot(torch.tensor(1), 2)

    return label

def ToTensor(image):
    image = image.transpose((2,0,1)).astype(np.float32)
    image = torch.from_numpy(image)
    return image

In [5]:
# albumentations train

albumentations_train = albumentations.Compose([
    albumentations.Resize(height=256, width=256),
    albumentations.OneOf([
        albumentations.HorizontalFlip(p=0.8),
        albumentations.RandomRotate90(p=0.8),
        albumentations.VerticalFlip(p=0.8)
    ], p=1),
    albumentations.OneOf([
        albumentations.MotionBlur(p=0.75),
        albumentations.OpticalDistortion(p=0.75),
        albumentations.GaussNoise(p=0.75)
    ], p=1),
    albumentations.pytorch.ToTensorV2()
])

In [6]:
full_dataset = Dataset(transform = albumentations_train)

In [7]:
full_dataset[0]

(tensor([[[1.0000, 1.0000, 0.0000,  ..., 1.0000, 1.0000, 0.0000],
          [0.0000, 1.0000, 1.0000,  ..., 0.0000, 0.7128, 0.0000],
          [1.0000, 1.0000, 0.0000,  ..., 1.0000, 0.4569, 1.0000],
          ...,
          [1.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0000],
          [1.0000, 1.0000, 0.5446,  ..., 0.0000, 1.0000, 1.0000],
          [0.0022, 0.0000, 1.0000,  ..., 1.0000, 1.0000, 0.0000]],
 
         [[1.0000, 1.0000, 0.0000,  ..., 1.0000, 1.0000, 0.0000],
          [0.0000, 0.0000, 1.0000,  ..., 1.0000, 0.0000, 0.0000],
          [1.0000, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 1.0000],
          ...,
          [1.0000, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 1.0000,  ..., 1.0000, 0.0000, 1.0000]],
 
         [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 0.0000, 0.6073,  ..., 0.0000, 1.0000, 0.0000],
          [0.0000, 1.0000, 0.0000,  ...,

In [8]:
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_datset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

In [9]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
test_loader = DataLoader(test_datset, batch_size=4, shuffle=False, num_workers=0)

In [10]:
# model(ResNet50)

import torchvision.models.resnet as resnet
import torch.optim as optim

conv1x1 = resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock = resnet.BasicBlock

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=2, zero_init_residual=True):
        super(ResNet, self).__init__()
        self.inplanes = 32
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False) # 마찬가지로 전부 사이즈 조정
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 32, layers[0], stride=1) # 3 반복
        self.layer2 = self._make_layer(block, 64, layers[1], stride=2) # 4 반복
        self.layer3 = self._make_layer(block, 128, layers[2], stride=2) # 6 반복
        self.layer4 = self._make_layer(block, 256, layers[3], stride=2) # 3 반복
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256 * block.expansion, num_classes)

        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1): # planes -> 입력되는 채널 수
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion: 
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        # input [32, 128, 128] -> [C ,H, W]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        #x.shape =[32, 64, 64]

        x = self.layer1(x)
        #x.shape =[128, 64, 64]
        x = self.layer2(x)
        #x.shape =[256, 32, 32]
        x = self.layer3(x)
        #x.shape =[512, 16, 16]
        x = self.layer4(x)
        #x.shape =[1024, 8, 8]
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [11]:
resnet50 = ResNet(resnet.Bottleneck, [3,4,6,3], 2, True)

In [12]:
resnet50.cuda()

ResNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 

In [13]:
# 정의
optimizer = torch.optim.Adam(resnet50.parameters(), lr=1e-5)
loss_func = nn.CrossEntropyLoss().to(device)

In [14]:
from torchsummary import summary
summary(resnet50, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             864
       BatchNorm2d-2         [-1, 32, 128, 128]              64
              ReLU-3         [-1, 32, 128, 128]               0
         MaxPool2d-4           [-1, 32, 64, 64]               0
            Conv2d-5           [-1, 32, 64, 64]           1,024
       BatchNorm2d-6           [-1, 32, 64, 64]              64
              ReLU-7           [-1, 32, 64, 64]               0
            Conv2d-8           [-1, 32, 64, 64]           9,216
       BatchNorm2d-9           [-1, 32, 64, 64]              64
             ReLU-10           [-1, 32, 64, 64]               0
           Conv2d-11          [-1, 128, 64, 64]           4,096
      BatchNorm2d-12          [-1, 128, 64, 64]             256
           Conv2d-13          [-1, 128, 64, 64]           4,096
      BatchNorm2d-14          [-1, 128,

In [15]:
mode = 'train'

In [16]:
dataiter = iter(train_loader)
image, labels = dataiter.next()

In [17]:
# 이미지 그리드 만들기
img_grid = torchvision.utils.make_grid(image)
writer.add_image('inhovation_images', img_grid)

In [18]:
# 모델 구조 올리기
writer.add_graph(resnet50, image.cuda().float())
writer.close()

In [19]:
# 전이 학습
resnet50 = torchvision.models.resnet50(pretrained=True).to(device)

In [20]:
resnet50.fc = nn.Linear(resnet50.fc.in_features, 2).to(device)

In [21]:
summary(resnet50, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [29]:
from pylab import *

In [25]:
def images_to_probs(net, images):
    '''
    학습된 신경망과 이미지 목록으로부터 예측 결과 및 확률을 생성합니다
    '''
    output = net(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.cpu().numpy())
    return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]

In [23]:
def plot_classes_preds(net, images, labels):
    '''
    학습된 신경망과 배치로부터 가져온 이미지 / 라벨을 사용하여 matplotlib
    Figure를 생성합니다. 이는 신경망의 예측 결과 / 확률과 함께 정답을 보여주며,
    예측 결과가 맞았는지 여부에 따라 색을 다르게 표시합니다. "images_to_probs"
    함수를 사용합니다.
    '''
    preds, probs = images_to_probs(net, images)
    # 배치에서 이미지를 가져와 예측 결과 / 정답과 함께 표시(plot)합니다
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx], one_channel=True)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            classes[preds[idx]],
            probs[idx] * 100.0,
            classes[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig

In [31]:
# train
loss_arr = []
if mode == 'train':
    for epoch in range(0 , 100):
        resnet50.train()
        loss_arr = []
        for batch, data in enumerate(train_loader, 0):
            inputs, labels = data # input data, label 분리
            inputs = inputs.to(device).float()
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = resnet50(inputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            
            loss_arr += [loss.item()]
            
       # writer.add_scalar('trainin loss', np.mean(loss_arr), epoch * len(train_loader)+ batch)
        #writer.add_figure('predictions vs. actuals', plot_classes_preds(resnet50, inputs, labels),
                         #gloabal_step = epoch * len(train_loader) + batch)
        print("TRAIN: EPOCH %04d / %04d | BATCH %04d / %04d | LOSS %.4f" %
                  (epoch, 100, batch, 135, np.mean(loss_arr)))   
        loss_arr = []
print('Finish!!')

TRAIN: EPOCH 0000 / 0100 | BATCH 0134 / 0135 | LOSS 0.6809
TRAIN: EPOCH 0001 / 0100 | BATCH 0134 / 0135 | LOSS 0.6774
TRAIN: EPOCH 0002 / 0100 | BATCH 0134 / 0135 | LOSS 0.6807
TRAIN: EPOCH 0003 / 0100 | BATCH 0134 / 0135 | LOSS 0.6784
TRAIN: EPOCH 0004 / 0100 | BATCH 0134 / 0135 | LOSS 0.6774
TRAIN: EPOCH 0005 / 0100 | BATCH 0134 / 0135 | LOSS 0.6761
TRAIN: EPOCH 0006 / 0100 | BATCH 0134 / 0135 | LOSS 0.6808
TRAIN: EPOCH 0007 / 0100 | BATCH 0134 / 0135 | LOSS 0.6816
TRAIN: EPOCH 0008 / 0100 | BATCH 0134 / 0135 | LOSS 0.6772
TRAIN: EPOCH 0009 / 0100 | BATCH 0134 / 0135 | LOSS 0.6827
TRAIN: EPOCH 0010 / 0100 | BATCH 0134 / 0135 | LOSS 0.6774
TRAIN: EPOCH 0011 / 0100 | BATCH 0134 / 0135 | LOSS 0.6808
TRAIN: EPOCH 0012 / 0100 | BATCH 0134 / 0135 | LOSS 0.6789
TRAIN: EPOCH 0013 / 0100 | BATCH 0134 / 0135 | LOSS 0.6810
TRAIN: EPOCH 0014 / 0100 | BATCH 0134 / 0135 | LOSS 0.6798
TRAIN: EPOCH 0015 / 0100 | BATCH 0134 / 0135 | LOSS 0.6775
TRAIN: EPOCH 0016 / 0100 | BATCH 0134 / 0135 | LOSS 0.67

In [35]:
resnet50.eval()
correct = 0
total = 0
test_loss = 0
acc = []
criterion= nn.CrossEntropyLoss().to(device)
for batch, data in enumerate(test_loader, 0):
    inputs, labels = data # input data, label 분리
    inputs = inputs.to(device).float()
    labels = labels.to(device)
    output = resnet50(inputs)
    _, predicted = torch.max(output.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    test_loss += criterion(output, labels).item()
    acc.append(100 * correct/total)

    print('\nTest set : Average loss:{:.4f}, Accuracy: {}/{}({:.0f}%)\n'.format(
                      test_loss, correct, total, 100 * correct/total
                  ))


Test set : Average loss:0.7168, Accuracy: 1/4(25%)


Test set : Average loss:1.3557, Accuracy: 3/8(38%)


Test set : Average loss:2.0192, Accuracy: 5/12(42%)


Test set : Average loss:2.7754, Accuracy: 6/16(38%)


Test set : Average loss:3.4275, Accuracy: 8/20(40%)


Test set : Average loss:4.0885, Accuracy: 10/24(42%)


Test set : Average loss:4.7204, Accuracy: 12/28(43%)


Test set : Average loss:5.4345, Accuracy: 13/32(41%)


Test set : Average loss:6.1056, Accuracy: 15/36(42%)


Test set : Average loss:6.6886, Accuracy: 18/40(45%)


Test set : Average loss:7.3275, Accuracy: 21/44(48%)


Test set : Average loss:8.0454, Accuracy: 23/48(48%)


Test set : Average loss:8.8742, Accuracy: 24/52(46%)


Test set : Average loss:9.6268, Accuracy: 26/56(46%)


Test set : Average loss:10.3377, Accuracy: 28/60(47%)


Test set : Average loss:11.1030, Accuracy: 28/64(44%)


Test set : Average loss:11.7605, Accuracy: 30/68(44%)


Test set : Average loss:12.4134, Accuracy: 32/72(44%)


Test set : A